In [1]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

# modelo_nome = 'modelo_tenis_multiclasse_braços.h5'
modelo_nome = 'modelo_tenis_multiclasse_v2.h5'

## Carregar modelo treinado

In [2]:
#Carregar modelo
model = tf.keras.models.load_model(modelo_nome)

#Inicia MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

landmarks_seq = []

video_path = "./videos_teste/fh_teste.mov"
cap = cv2.VideoCapture(video_path)

!['KeyPoints HPE'](KeyPoints_HPE.jpg)

## Usar um vídeo como referência

In [ ]:
BRAÇO_LANDMARKS = [11, 12, 13, 14, 15, 16]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        frame_landmarks = []
        
        #Captura apenas os 6 pontos dos braços
        for i in BRAÇO_LANDMARKS:
            lm = results.pose_landmarks.landmark[i]
            frame_landmarks.extend([lm.x, lm.y, lm.z])

        landmarks_seq.append(frame_landmarks)

cap.release()

#Padroniza para X frames
janela = 30
num_features = 18  #Apenas os pontos dos braços (6 landmarks x 3 coordenadas)

if len(landmarks_seq) < janela:
    padding = [[0] * num_features] * (janela - len(landmarks_seq))  #Garante o formato correto (1, 30, 18)
    landmarks_seq = padding + landmarks_seq
else:
    landmarks_seq = landmarks_seq[-janela:]

#Agora a entrada sempre terá (1, 30, 18)
X_input = np.array([landmarks_seq])

print(f"Shape de X_input antes da predição: {X_input.shape}")

#Fazer a previsão corretamente
predictions = model.predict(X_input)[0]  #Obtém as probabilidades das 3 classes

#Obtém a classe com maior probabilidade
classe_predita = np.argmax(predictions)
labels = ["Forehand", "Backhand", "Nenhum"]
movimento = labels[classe_predita]  #Converte índice para nome da classe

print(f"Forehand: {predictions[0]:.4f} \nBackhand: {predictions[1]:.4f} \nNenhum: {predictions[2]:.4f}")
print(f"Movimento detectado: {movimento}")

Shape de X_input antes da predição: (1, 30, 18)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
Movimento detectado: Forehand
Probabilidades -> Forehand: 0.9977 | Backhand: 0.0021 | Nenhum: 0.0002


## Teste em tempo real

In [5]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import time
from collections import deque

#Abrir câmera em tempo real
cap = cv2.VideoCapture(0)
#Para arrumar o FPS da câmera
camera_fps = 30

mp_drawing = mp.solutions.drawing_utils

cap.set(cv2.CAP_PROP_FPS, camera_fps)

#Cache que armazena os últimos X frames
frames_cache = 30
landmarks_seq = deque(maxlen=frames_cache)  

labels = ["Forehand", "Backhand", "Nenhum"]

braco_landmarks = [11, 12, 13, 14, 15, 16]

#Variáveis que controlam o título
movimento_atual = "Nenhum"
ultima_classificacao = time.time()
cor_movimento = (0, 255, 0)

#Contadores de movimentos
forehand_count = 0
backhand_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        #Para extrair somente os landmarks dos braços
        frame_landmarks = []
        for i in braco_landmarks:
            lm = results.pose_landmarks.landmark[i]
            frame_landmarks.extend([lm.x, lm.y, lm.z])

        #Adicionar os landmarks ao buffer (mantém apenas os últimos X frames)
        landmarks_seq.append(frame_landmarks)

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    #Faz a previsão constantemente
    if len(landmarks_seq) >= 30:  #Verifica a cada X frames
        X_input = np.array([list(landmarks_seq)[-30:]])  #Últimos 30 frames

        # Fazer a previsão
        predictions = model.predict(X_input)[0]  #Obtém as 3 probabilidades
        classe_predita = np.argmax(predictions)  #Obtém a classe com maior probabilidade
        confianca = predictions[classe_predita]  #Obtém a confiança da previsão

        #Aplicar um limiar de confiança para evitar previsões erradas
        if confianca < 0.5:
            movimento = "Nenhum"
        else:
            movimento = labels[classe_predita]

        #Se o movimento for Forehand ou Backhand, segurar o título por 1 segundo
        if movimento in ["Forehand", "Backhand"]:
            if movimento != movimento_atual:
                movimento_atual = movimento
                ultima_classificacao = time.time()
                cor_movimento = (255, 0, 0)  # Azul durante 1 segundo

                #Aumenta o contador correspondente
                if movimento == "Forehand":
                    forehand_count += 1
                elif movimento == "Backhand":
                    backhand_count += 1

        #Se já passou 1 segundo desde a última classificação, voltar para verde
        if time.time() - ultima_classificacao > 1:
            cor_movimento = (0, 255, 0)  # Verde novamente

        #Exibir o resultado na tela
        cv2.putText(frame, f"Movimento: {movimento_atual}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, cor_movimento, 2)

        #Exibir as probabilidades no console (para depuração)
        print(f"Forehand: {predictions[0]:.4f} | Backhand: {predictions[1]:.4f} | Nenhum: {predictions[2]:.4f}")

        pred_forehand = predictions[0] * 100
        pred_backhand = predictions[1] * 100

        cv2.putText(frame, f"Forehand: {(pred_forehand):.2f}%",
                    (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, cor_movimento, 2)
        cv2.putText(frame, f"Backhand: {(pred_backhand):.2f}%",
                    (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, cor_movimento, 2)
        # cv2.putText(frame, f"Nenhum: {predictions[2]:.4f}",
        #             (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, cor_movimento, 2)

        #Exibir contadores de movimentos na tela
        cv2.putText(frame, f"Forehand : {forehand_count}", (50, 250),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(frame, f"Backhand : {backhand_count}", (50, 300),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    cv2.imshow("Detecção em Tempo Real", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  #Reduz delay para capturar mais frames
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Forehand: 0.0026 | Backhand: 0.1034 | Nenhum: 0.8940
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Forehand: 0.0026 | Backhand: 0.1033 | Nenhum: 0.8941
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Forehand: 0.0026 | Backhand: 0.1032 | Nenhum: 0.8941
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Forehand: 0.0026 | Backhand: 0.1028 | Nenhum: 0.8946
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Forehand: 0.0027 | Backhand: 0.1022 | Nenhum: 0.8951
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Forehand: 0.0027 | Backhand: 0.1019 | Nenhum: 0.8953
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Forehand: 0.0027 | Backhand: 0.1005 | Nenhum: 0.8968
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Forehand: 0.0027 | Backhand: 0.0985 | Nenhum: 0.8987
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Forehand: 0.0028 | Backhand: 0.0966 | Nenhum: 0.9006
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Forehand: 0.0029 | Backhand: 0.0949 | Nenhum: 0.9023
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Forehand: 0.0030 | Backhand: 0.0927 | Nenhum: 0.9044